# Verify annotation values

In [1]:
import glob
import pympi


empty_annotation_value = ['']
food_annotation_values = ['fork', 'knife', 'spoon', 'chopsticks', 'hand']
drink_annotation_values = ['cup', 'bottle']
disruption_annotation_values = ['light_off', 'participant_left']

valid_annotation_values = {
    'mouth_open': empty_annotation_value,

    'food_to_mouth': food_annotation_values, # can also have an empty annotation value - there is only one special case (see below and/or annotation instructions)
    'food_entered': food_annotation_values,
    'food_lifted': food_annotation_values, # can also have an empty annotation value - there is only one special case (see below and/or annotation instructions)

    'drink_to_mouth': drink_annotation_values,
    'drink_entered': drink_annotation_values,
    'drink_lifted': drink_annotation_values,

    'napkin_to_mouth': empty_annotation_value,
    'napkin_entered': empty_annotation_value,
    'napkin_lifted': empty_annotation_value,

    'disruption': disruption_annotation_values
}

annotations_folder = f'/home/janko/Desktop/social-dining/data/annotation/annotation-files'

for annotation_file in sorted(glob.glob(f'{annotations_folder}/*.eaf')):
    eaf_obj1 = pympi.Elan.Eaf(annotation_file)
    available_tier_names = eaf_obj1.get_tier_names()

    for tier_name, valid_values in valid_annotation_values.items():
        if tier_name not in available_tier_names:
            print(f"WARNING: no '{tier_name}' tier name found in file: {annotation_file}")
        else:
            for annotation in eaf_obj1.get_annotation_data_for_tier(tier_name):
                # assert annotation[2] in valid_values, f"ERROR: the annotation {annotation} with tier name '{tier_name}' must have a value from {valid_values}. Found in file: {annotation_file}"
                if annotation[2] not in valid_values:
                    print(f"ERROR: the annotation {annotation} with tier name '{tier_name}' must have a value from {valid_values}. Found in file: {annotation_file}")

# Fixed errors (6):
    # AssertionError: ERROR: the annotation (1642498, 1646371, '') with tier name 'drink_to_mouth' must have a value from ['cup', 'bottle']. Found in file: /home/janko/Desktop/social-dining/data/annotation/annotation-files/05_3.eaf
    # AssertionError: ERROR: the annotation (1141239, 1141639, 'hand') with tier name 'drink_entered' must have a value from ['cup', 'bottle']. Found in file: /home/janko/Desktop/social-dining/data/annotation/annotation-files/07_2.eaf
    # Immediately next to AssertionError: ERROR: the annotation (1141239, 1141639, 'hand') with tier name 'drink_entered' must have a value from ['cup', 'bottle']. Found in file: /home/janko/Desktop/social-dining/data/annotation/annotation-files/07_2.eaf
    # AssertionError: ERROR: the annotation (2828970, 2829370, 'fork') with tier name 'drink_lifted' must have a value from ['cup', 'bottle']. Found in file: /home/janko/Desktop/social-dining/data/annotation/annotation-files/08_1.eaf
    # AssertionError: ERROR: the annotation (1105737, 1106447, '') with tier name 'food_to_mouth' must have a value from ['fork', 'knife', 'spoon', 'chopsticks', 'hand']. Found in file: /home/janko/Desktop/social-dining/data/annotation/annotation-files/09_3.eaf
    # AssertionError: ERROR: the annotation (510900, 511300, 'hand') with tier name 'drink_lifted' must have a value from ['cup', 'bottle']. Found in file: /home/janko/Desktop/social-dining/data/annotation/annotation-files/22_2.eaf

# Only one special case that is accepted (the person ate without the use of any utensil/hand and the person just moved head towards the table to eat a bite)
    # ERROR: the annotation (2795170, 2795650, '') with tier name 'food_to_mouth' must have a value from ['fork', 'knife', 'spoon', 'chopsticks', 'hand']. Found in file: /home/janko/Desktop/social-dining/data/annotation/annotation-files/29_1.eaf
    # ERROR: the annotation (2793518, 2793918, '') with tier name 'food_lifted' must have a value from ['fork', 'knife', 'spoon', 'chopsticks', 'hand']. Found in file: /home/janko/Desktop/social-dining/data/annotation/annotation-files/29_1.eaf


ERROR: the annotation (2795170, 2795650, '') with tier name 'food_to_mouth' must have a value from ['fork', 'knife', 'spoon', 'chopsticks', 'hand']. Found in file: /home/janko/Desktop/social-dining/data/annotation/annotation-files/29_1.eaf
ERROR: the annotation (2793518, 2793918, '') with tier name 'food_lifted' must have a value from ['fork', 'knife', 'spoon', 'chopsticks', 'hand']. Found in file: /home/janko/Desktop/social-dining/data/annotation/annotation-files/29_1.eaf


# Verify annotation order and count

In [1]:
import glob
import pympi

tier_names = [
    'mouth_open',
    'food_lifted',
    'food_to_mouth',

    'drink_lifted',
    'drink_to_mouth',

    'napkin_lifted',
    'napkin_to_mouth'
]
annotations_folder = f'/home/janko/Desktop/social-dining/data/annotation/annotation-files'
# Milliseconds to minutes
MTM = 60000

for annotation_file in sorted(glob.glob(f'{annotations_folder}/*.eaf')):
    print(f"In file: {annotation_file}")
    eaf_obj1 = pympi.Elan.Eaf(annotation_file)
    available_tier_names = eaf_obj1.get_tier_names()

    annotations = {}
    for tier_name in tier_names:
        if tier_name not in available_tier_names:
            print(f"\t WARNING: no '{tier_name}' tier name found")
        else:
            annotations[tier_name] = sorted(eaf_obj1.get_annotation_data_for_tier(tier_name), key=lambda x: x[0])

    # The food_lifted annotation must precede the food_to_mouth annotation unless the food is already lifted at the beginning of the video
    # The mouth_open annotation must precede the food_to_mouth annotation
    print(f"\t #food_lifted annotations: {len(annotations['food_lifted'])}\t #mouth_open annotations: {len(annotations['mouth_open'])}\t #food_to_mouth annotations: {len(annotations['food_to_mouth'])}")
    assert len(annotations['mouth_open']) == len(annotations['food_to_mouth']), f"\t ERROR: the number of 'mouth_open' annotations {len(annotations['mouth_open'])} must be equal to the number of 'food_to_mouth' annotations {len(annotations['food_to_mouth'])}"
    i1, i2 = 0, 0
    while i2 < len(annotations['food_to_mouth']):
        if i1 == 0 and annotations['food_lifted'][i1][0] > annotations['food_to_mouth'][i2][0]:
            print(f"\t WARNING: food already lifted at the beginning of the video")
        else:
            assert annotations['food_lifted'][i1][0] <= annotations['food_to_mouth'][i2][0], f"\t ERROR: the 'food_lifted' annotation at {annotations['food_lifted'][i1][0]/MTM} does not precede the 'food_to_mouth' annotation at {annotations['food_to_mouth'][i2][0]/MTM}"
            i1 += 1
        assert annotations['mouth_open'][i2][0] <= annotations['food_to_mouth'][i2][0], f"\t ERROR: the 'mouth_open' annotation at {annotations['mouth_open'][i2][0]/MTM} does not precede the 'food_to_mouth' annotation at {annotations['food_to_mouth'][i3][0]/MTM}"
        i2 += 1

    # The drink_lifted annotation must precede the drink_to_mouth annotation unless the drink is already lifted at the beginning of the video
    print(f"\t #drink_lifted annotations: {len(annotations['drink_lifted'])}\t #drink_to_mouth annotations: {len(annotations['drink_to_mouth'])}")
    i1, i2 = 0, 0
    while i2 < len(annotations['drink_to_mouth']):
        if i1 == 0 and annotations['drink_lifted'][i1][0] > annotations['drink_to_mouth'][i2][0]:
            print(f"\t WARNING: drink already lifted at the beginning of the video")
        else:
            assert annotations['drink_lifted'][i1][0] <= annotations['drink_to_mouth'][i2][0], f"ERROR: the 'drink_lifted' annotation at {annotations['drink_lifted'][i1][0]/MTM} does not precede the 'drink_to_mouth' annotation at {annotations['drink_to_mouth'][i2][0]/MTM}"
            i1 += 1
        i2 += 1

    # The napkin_lifted annotation must precede the napkin_to_mouth annotation unless the napkin is already lifted at the beginning of the video.    
    print(f"\t #napkin_lifted annotations: {len(annotations['napkin_lifted'])}\t #napkin_to_mouth annotations: {len(annotations['napkin_to_mouth'])}")
    i1, i2 = 0, 0
    while i2 < len(annotations['napkin_to_mouth']):
        if i1 == 0 and annotations['napkin_lifted'][i1][0] > annotations['napkin_to_mouth'][i2][0]:
            print(f"\t WARNING: napkin already lifted at the beginning of the video")
        else:
            assert annotations['napkin_lifted'][i1][0] <= annotations['napkin_to_mouth'][i2][0], f"ERROR: the 'napkin_lifted' annotation at {annotations['napkin_lifted'][i1][0]/MTM} does not precede the 'napkin_to_mouth' annotation at {annotations['napkin_to_mouth'][i2][0]/MTM}"
            i1 += 1
        i2 += 1

    print('=' * 100)


In file: /home/janko/Desktop/social-dining/data/annotation/annotation-files/01_1.eaf
	 #food_lifted annotations: 38	 #mouth_open annotations: 38	 #food_to_mouth annotations: 38
	 #drink_lifted annotations: 12	 #drink_to_mouth annotations: 12
	 #napkin_lifted annotations: 14	 #napkin_to_mouth annotations: 14
In file: /home/janko/Desktop/social-dining/data/annotation/annotation-files/01_2.eaf
	 #food_lifted annotations: 79	 #mouth_open annotations: 79	 #food_to_mouth annotations: 79
	 #drink_lifted annotations: 7	 #drink_to_mouth annotations: 7
	 #napkin_lifted annotations: 0	 #napkin_to_mouth annotations: 0
In file: /home/janko/Desktop/social-dining/data/annotation/annotation-files/01_3.eaf
	 #food_lifted annotations: 66	 #mouth_open annotations: 66	 #food_to_mouth annotations: 66
	 #drink_lifted annotations: 9	 #drink_to_mouth annotations: 9
	 #napkin_lifted annotations: 0	 #napkin_to_mouth annotations: 0
In file: /home/janko/Desktop/social-dining/data/annotation/annotation-files/02_1.

# Get values to unify disruption period start and end times across videos from one session

After running this script, manually change the few ELAN files with the min and max values.

In [4]:
import glob
import pympi

annotations_folder = f'/home/janko/Desktop/social-dining/data/annotation/annotation-files'

i = 0
for annotation_file in sorted(glob.glob(f'{annotations_folder}/*.eaf')):
    eaf_obj1 = pympi.Elan.Eaf(annotation_file)

    x = eaf_obj1.get_annotation_data_for_tier('disruption')
    if len(x) > 0:
        print(annotation_file, x)
        if x[0][2] == 'light_off':
            i += 1
            if i == 1:
                min_start_time = x[0][0]
                max_end_time = x[0][1]
            elif i == 2:
                min_start_time = min(min_start_time, x[0][0])
                max_end_time = max(max_end_time, x[0][1])
            elif i == 3:
                min_start_time = min(min_start_time, x[0][0])
                max_end_time = max(max_end_time, x[0][1])

                print(f'Minimum start time: {min_start_time}')
                print(f'Maximum end time: {max_end_time}')
                i = 0
        

/home/janko/Desktop/social-dining/data/annotation/annotation-files/09_1.eaf [(1562454, 1584669, 'light_off')]
/home/janko/Desktop/social-dining/data/annotation/annotation-files/09_2.eaf [(1562454, 1584669, 'light_off')]
/home/janko/Desktop/social-dining/data/annotation/annotation-files/09_3.eaf [(1562454, 1584669, 'light_off')]
Minimum start time: 1562454
Maximum end time: 1584669
/home/janko/Desktop/social-dining/data/annotation/annotation-files/11_3.eaf [(1467894, 1554839, 'participant_left')]
/home/janko/Desktop/social-dining/data/annotation/annotation-files/15_1.eaf [(942016, 974579, 'light_off')]
/home/janko/Desktop/social-dining/data/annotation/annotation-files/15_2.eaf [(942016, 974579, 'light_off')]
/home/janko/Desktop/social-dining/data/annotation/annotation-files/15_3.eaf [(942016, 974579, 'light_off')]
Minimum start time: 942016
Maximum end time: 974579
/home/janko/Desktop/social-dining/data/annotation/annotation-files/18_1.eaf [(1262789, 1292636, 'light_off')]
/home/janko/D